## *Настройка витрины данных на основе существующей схемы PostgreSQL*
### Витрина данных (Data Mart) создается для удобного анализа и агрегации данных. Мы будем использовать SQL-запросы для построения витрины на основе уже загруженных таблиц:

- orders (заказы)
- products_category (категории товаров)
- order_items (позиции заказов)
1. Определение структуры витрины
### Витрина должна объединять данные из всех трех таблиц и содержать:

- Информацию о заказах (дата покупки, дата доставки, ID клиента)
- Информацию о товарах (категория)
- Финансовые показатели (цена, стоимость доставки, суммарная выручка)
- Метрики по доставке (время доставки)
### Создадим витрину order_summary в PostgreSQL.



### Установка и подключение библиотек

In [1]:
%pip install pandas sqlalchemy psycopg2 matplotlib

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

### Подключение к PostgreSQL

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Параметры подключения к базе данных
db_user = "myuser"
db_password = "mypassword"
db_host = "localhost"
db_port = "5432"
db_name = "mydatabase"

connection_string = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# Загрузка данных из CSV
try:
    products_category = pd.read_csv('products_category.csv')
    order_items = pd.read_csv('order_items.csv')
    orders = pd.read_csv('orders.csv')

    # Запись данных в PostgreSQL
    products_category.to_sql("products_category", engine, if_exists="replace", index=False, method='multi')
    order_items.to_sql("order_items", engine, if_exists="replace", index=False, method='multi')
    orders.to_sql("orders", engine, if_exists="replace", index=False, method='multi')

    print("Импорт завершён!")
except Exception as e:
    print(f"Произошла ошибка: {e}")

### Создание витрины данных

In [ ]:
query = """
CREATE MATERIALIZED VIEW IF NOT EXISTS order_summary AS
SELECT 
    o.order_id,
    o.customer_id,
    o.purchase_timestamp,
    o.delivered_customer_date,
    p.product_category_name,
    oi.price,
    oi.sales_value,
    oi.freight_value,
    oi.delivery_time,
    (oi.sales_value + oi.freight_value) AS total_revenue
FROM order_items oi
JOIN orders o ON oi.order_id = o.order_id
JOIN products_category p ON oi.product_id = p.product_id;
"""

with engine.connect() as conn:
    conn.execute(query)
    print("Витрина order_summary создана!")

### Обновление витрины (при необходимости)

In [ ]:
with engine.connect() as conn:
    conn.execute("REFRESH MATERIALIZED VIEW order_summary;")
    print("Витрина обновлена!")

### Загрузка данных из витрины

In [ ]:
df = pd.read_sql("SELECT * FROM order_summary", con=engine)
df.head()

### Визуализация данных

In [ ]:
category_sales = df.groupby('product_category_name')['total_revenue'].sum()
category_sales.sort_values(ascending=False).plot(kind='bar', figsize=(10,5), title="Выручка по категориям товаров")
plt.ylabel("Выручка")
plt.show()